## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## 確認硬體資源 (如果你是在 Linux, 若是在 Windows, 請參考 https://blog.csdn.net/idwtwt/article/details/78017565)
!nvidia-smi

'nvidia-smi' 不是內部或外部命令、可執行的程式或批次檔。


In [2]:
import os
import keras

os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [3]:
# 從 Keras 的內建功能中，取得 train 與 test 資料集
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
"""
Your code here
"""
def preproc_x(x, flatten=True):
    x = (x - x.min()) / (x.max() - x.min()) * 2 - 1
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
"""Code Here
建立你的神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 64, 32]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer4 (Dense)        (None, 64)                8256      
_________________________________________________________________
hidden_layer5 (Dense)        (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 10)                330 

In [7]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [8]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [9]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 35s 692us/step - loss: 1.7279 - accuracy: 0.3864 - val_loss: 1.5655 - val_accuracy: 0.4429
Epoch 2/100
50000/50000 [==============================] - 35s 702us/step - loss: 1.4816 - accuracy: 0.4788 - val_loss: 1.4601 - val_accuracy: 0.4873
Epoch 3/100
50000/50000 [==============================] - 34s 677us/step - loss: 1.3586 - accuracy: 0.5224 - val_loss: 1.4001 - val_accuracy: 0.5076
Epoch 4/100
50000/50000 [==============================] - 28s 569us/step - loss: 1.2646 - accuracy: 0.5560 - val_loss: 1.3867 - val_accuracy: 0.5112
Epoch 5/100
50000/50000 [==============================] - 30s 594us/step - loss: 1.1831 - accuracy: 0.5807 - val_loss: 1.3805 - val_accuracy: 0.5102
Epoch 6/100
50000/50000 [==============================] - 38s 756us/step - loss: 1.1018 - accuracy: 0.6105 - val_loss: 1.3669 - val_accuracy: 0.5281
Epoch 7/100
50000/50000 [=========================

Epoch 55/100
50000/50000 [==============================] - 29s 585us/step - loss: 0.0976 - accuracy: 0.9679 - val_loss: 4.0294 - val_accuracy: 0.5309
Epoch 56/100
50000/50000 [==============================] - 29s 585us/step - loss: 0.0859 - accuracy: 0.9723 - val_loss: 4.1733 - val_accuracy: 0.5209
Epoch 57/100
50000/50000 [==============================] - 30s 598us/step - loss: 0.1000 - accuracy: 0.9673 - val_loss: 4.1976 - val_accuracy: 0.5203
Epoch 58/100
50000/50000 [==============================] - 32s 642us/step - loss: 0.0969 - accuracy: 0.9679 - val_loss: 4.2453 - val_accuracy: 0.5169
Epoch 59/100
50000/50000 [==============================] - 33s 662us/step - loss: 0.0874 - accuracy: 0.9722 - val_loss: 4.0813 - val_accuracy: 0.5289
Epoch 60/100
50000/50000 [==============================] - 33s 659us/step - loss: 0.1226 - accuracy: 0.9597 - val_loss: 4.0806 - val_accuracy: 0.5251
Epoch 61/100
50000/50000 [==============================] - 33s 659us/step - loss: 0.1070 - ac

In [10]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>